# Data Discovery Demonstration

The **GOAL** of *data discovery* is to find and suggest datasets that are relevant to answer a question. 

##### Scenario:
* Users of the *discovery system* interact, during a *data discovery session* to explore the available datasets in the organization.
* Users can choose among different **primitives** that capture user's intuition. Examples: "give me any column that contains this term", i.e. keyword search, "I want to see all columns that are *similar* to this one".

##### Demo Setup
* Small dataset (50MB) that consists of 10 files extracted from data.gov. 
* Data related to: (i) taxes, (ii) taxes for liquor and tobacco and (iii) contracts granted to institutions such as grants, etc. 


#### Setting
* Server that exposes an API (also can be used as embedded libraries, or as a plugin for ElasticSearch, Splunk, etc).
* We do not own data. We have read access to required datasets.
* Systems serves while ingests new data and previously ingested data that is updated.

##### Pre-Processing: Loading data and creating signatures

In [9]:
import api as API
API.load_precomputed_model()

Loading signature collections...
Done deserialization of signature collections!
Loading signature collections...DONE!
Loading graph...
Done deserialization of signature collection!
Loading graph...DONE!
Loading simrank matrix...
Done deserialization of simrank matrix!
Loading simrank matrix...DONE!
Loading dataset columns...
Done deserialization of dataset columns!
Loading dataset columns...DONE!


## Data Discovery Primitives

We present 5 different primitives for data discovery (One important goal is to *understand what is the minimum set of primitives necessary to approach data discovery*.)
1. **Keyword search** : Traditional keyword search on all the datasets available.
2. **Column similarity** : Provide all columns similar to column X (text similarity or distribution similarity in the case of numerical columns).
3. **Structural similarity** : Provide all columns similar to column X, where similarity captures structural knowledge (i.e. common neighbors).
4. **Column-Pair similarity** : Given a pair of columns, find other pairs that are related with a similar column.
5. **Relation similarity** : Given X,Y, find f(X->Y). Then find Z such that f is similar to s, where s is s(X->Z)

We demonstrate 3, **column similarity**, **structural similarity** and **column-pair similarity**.

### Column Similarity

The column similarity primitive will return all columns in the dataset that are similar to the one provided. In the case of textual columns, a similar column is one that contains similar term distribution and content. In the case of numerical columns, a similar column is one with similar data distribution.

---

***Example A***: Which columns refer to US states? Find columns similar to "Geography" which is a **textual** column.

In [10]:
filename = "STC_2014_STC005_with_ann.csv"
columnname = "Geography"
res = API.columns_similar_to(filename, columnname, "ks")
API.print_result(res)

**datafeeds\2000_NASA_Contracts_Full_20151015.csv**

   statecode


**STC_2014_STC005_with_ann.csv**

   Geography


**SAMHSA_Synar_Reports__Youth_Tobacco_Sales.csv**

...

   City


**The_Tax_Burden_on_Tobacco_Volume_49__1970-2014.csv**

   LocationDesc


**STC_2014_STC006.US01_with_ann.csv**

   Government


---

***Example B***: What columns in the dataset are taxes? Find columns similar to "License Taxes - Alcoholic Beverages License" which is a **numerical** column

In [11]:
filename = "STC_2014_STC005_with_ann.csv"
columname = "License Taxes - Alcoholic Beverages License"
res = API.columns_similar_to(filename, columname, "ks")
API.print_result(res)

**STC_2014_STC005_with_ann.csv**

   License Taxes - Other License Taxes
   Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Amusements Sales Tax
   License Taxes - Alcoholic Beverages License


### Structural Similarity

The structural similarity primitive will return any column that is *structurally similar* to the one provided. Structural similarity relies on the structural context in which relations between objects occur, and captures relationships among such objects. Our primitive relies on a graph where nodes are columns and there are edges whenever the similarity between two nodes is higher than a given threshold. On such graph, we run SimRank to compute structural similarity and answer operations that require this primitive.

In [12]:
concept = ("datafeeds\\2000_NASA_Contracts_Full_20151015.csv", "vendorname")
res = API.give_structural_sim_of(concept)
API.print_result(res)

**datafeeds\2000_NASA_Contracts_Full_20151015.csv**

   vendorlegalorganizationname
   vendordoingasbusinessname
   vendoralternatename
   vendorname
   mod_parent


**NYS_Liquor_Authority_New_Applications_Received.csv**

   Premises Name


**Contracts.csv**

   Vendor Name (USAspending)


**Contracts_matchUSASpending.csv**

   Vendor Name (USAspending)


In [ ]:
concept = ("Contracts_matchUSASpending.csv", "Agency Name")
res = API.give_structural_sim_of(concept)
API.print_result(res)

### The advantage of context

Column similarity will provide any column with a similarity higher than the threshold, when looking columns one to one. Structural similarity captures information about the context. For example ...

---
First we run **column similarity**

In [13]:
res = API.columns_similar_to("NYS_Liquor_Authority_New_Applications_Received.csv", "License Received Date", "ks")
API.print_result(res)

**NYS_Liquor_Authority_New_Applications_Received.csv**

   License Received Date


**Contracts.csv**

   Contract End Date (USAspending)
   Timestamp (Contract)
   Timestamp (Base Contract)
   Contract Start Date (USAspending)


If we run **structural similarity**

In [14]:
concept = ("NYS_Liquor_Authority_New_Applications_Received.csv", "License Received Date")
res = API.give_structural_sim_of(concept)
API.print_result(res)

**NYS_Liquor_Authority_New_Applications_Received.csv**

   License Received Date


### Column-Pair Similarity

Given a pair of columns X and Y that are related by some function f(X,Y), find other pairs of columns Z,W that are related similarly. Example. Given the correlation between X and Y, find other pairs of columns similarly correlated.

In [15]:
X = ("STC_2014_STC006.US01_with_ann.csv", "License Taxes")
Y = ("STC_2014_STC006.US01_with_ann.csv", "Other Taxes")
res = API.pairs_similar_to_pair(X, Y, "pearson")
API.print_result(res)

**('STC_2014_STC006.US01_with_ann.csv', 'License Taxes')**

   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Alcoholic Beverages License')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Corporations in General License')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Public Utilities Sales Tax')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Motor Vehicle Operators License')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Amusements Sales Tax')
   ('STC_2014_STC006.US01_with_ann.csv', 'Other Taxes')
   ('STC_2014_STC005_with_ann.csv', 'Other Taxes')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Occupation and Business License, NEC')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Rece

**('STC_2014_STC005_with_ann.csv', 'License Taxes - Other License Taxes')**

   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Alcoholic Beverages Sales Tax')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Insurance Premiums Sales Tax')
   ('STC_2014_STC005_with_ann.csv', 'Income Taxes')
   ('STC_2014_STC005_with_ann.csv', 'Total Taxes')
   ('STC_2014_STC006.US01_with_ann.csv', 'Income Taxes')
   ('STC_2014_STC006.US01_with_ann.csv', 'Sales and Gross Receipts Taxes')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Hunting and Fishing License')


**('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Insurance Premiums Sales Tax')**

...

   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Hunting and Fishing License')


**('STC_2014_STC005_with_ann.csv', 'License Taxes - Alcoholic Beverages License')**

   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Corporations in General License')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Public Utilities Sales Tax')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Motor Vehicle Operators License')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Amusements Sales Tax')
   ('STC_2014_STC006.US01_with_ann.csv', 'Other Taxes')
   ('STC_2014_STC005_with_ann.csv', 'Other Taxes')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Occupation and Business License, NEC')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Tobacco Products Sales Tax')
   ('STC_2014_STC006.US01_with_ann.csv', 'Total Taxes')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Re

**('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Tobacco Products Sales Tax')**

   ('STC_2014_STC006.US01_with_ann.csv', 'Total Taxes')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Other Selective Sales and Gross Receipts Taxes')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Other License Taxes')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Alcoholic Beverages Sales Tax')
   ('STC_2014_STC005_with_ann.csv', 'Sales and Gross Receipts Taxes - Selective Sales and Gross Receipts Taxes - Insurance Premiums Sales Tax')
   ('STC_2014_STC005_with_ann.csv', 'Income Taxes')
   ('STC_2014_STC005_with_ann.csv', 'Total Taxes')
   ('STC_2014_STC006.US01_with_ann.csv', 'Income Taxes')
   ('STC_2014_STC006.US01_with_ann.csv', 'Sales and Gross Receipts Taxes')
   ('STC_2014_STC005_with_ann.csv', 'License Taxes - Hunting and Fishing License')
